<a href="https://colab.research.google.com/github/heriyang9000/test2/blob/main/test1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **LIBRARY**

In [ ]:
pip install yfinance

In [ ]:
pip install progressbar2

In [ ]:
import getpass
import pandas as pd
import yfinance as yf
import panel as pn
import datetime
import requests
import datetime as dt
import numpy as np
import param
from bokeh.plotting import figure
from pandas_datareader import data as pdr
yf.pdr_override()
import progressbar

In [9]:
import zipfile
import os

In [11]:
!wget --no-check-certificate \
    "https://github.com/heriyang9000/mysys/archive/refs/heads/main.zip" \
    -O "/tmp/cats-and-dogs.zip"


zip_ref = zipfile.ZipFile('/tmp/cats-and-dogs.zip', 'r') #Opens the zip file in read mode
zip_ref.extractall('/tmp') #Extracts the files into the /tmp folder
zip_ref.close()

--2021-11-19 12:51:00--  https://github.com/heriyang9000/mysys/archive/refs/heads/main.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/heriyang9000/mysys/zip/refs/heads/main [following]
--2021-11-19 12:51:00--  https://codeload.github.com/heriyang9000/mysys/zip/refs/heads/main
Resolving codeload.github.com (codeload.github.com)... 140.82.112.9
Connecting to codeload.github.com (codeload.github.com)|140.82.112.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘/tmp/cats-and-dogs.zip’

/tmp/cats-and-dogs.     [  <=>               ]  69.40M  5.85MB/s    in 15s     

2021-11-19 12:51:15 (4.70 MB/s) - ‘/tmp/cats-and-dogs.zip’ saved [72767675]



In [ ]:
print('Test set dogs' , len(os.listdir('dog-cat-full-dataset-master/data/test/dogs')))

## **API** **KEY**

In [7]:
api_key = getpass.getpass('Masukan Api Key Anda lalu klik enter? ')
response = requests.get(f"https://api.quantxi.com/user?api_key={api_key}")
while response.json()['status'] == 'failed':
  print("Maaf API Key yang anda masukkan salah, coba di ulang lagi !!!")
  api_key = getpass.getpass('Masukan Api Key Anda lalu klik enter? ')
  response = requests.get(f"https://api.quantxi.com/user?api_key={api_key}")
  if response.json()['status'] == 'success':
    print(f"Selamat Datang {response.json()['data']['nama']}")


KeyboardInterrupt: ignored

# **SETTING**

In [ ]:
initial_euqity = int(input('Masukan Initial Equity Anda, Min= 1,000,000 Max= 100,000,000 lalu klik enter? '))
while initial_euqity < 1000000 or initial_euqity > 100000000:
  print("Maaf initial_euqity yang anda masukkan salah, coba di ulang lagi !!!")
  initial_euqity = int(input('Masukan Initial Equity Anda, Min= 1,000,000 Max= 100,000,000 lalu klik enter? '))

spread = float(input('Masukan Spread+Slippage Anda, Min= 0.01 Max= 0.05 lalu klik enter? '))
while spread < 0.01 or spread > 0.05:
  print("Maaf spread yang anda masukkan salah, coba di ulang lagi !!!")
  spread = float(input('Masukan Spread+Slippage Anda, Min= 0.01 Max= 0.05 lalu klik enter? '))

commision = float(input('Masukan Commision Anda, Min= 0.01 Max= 0.05 lalu klik enter? '))
while commision < 0.01 or commision > 0.05:
  print("Maaf commision yang anda masukkan salah, coba di ulang lagi !!!")
  commision = float(input('Masukan Commision Anda, Min= 0.01 Max= 0.05 lalu klik enter? '))

interest = float(input('Masukan Interest Anda, Min= 0.01 Max= 0.05 lalu klik enter? '))
while interest < 0.01 or interest > 0.05:
  print("Maaf interest yang anda masukkan salah, coba di ulang lagi !!!")
  interest = float(input('Masukan Interest Anda, Min= 0.01 Max= 0.05 lalu klik enter? '))

print(f"Anda telah mensetting testing anda, Initial Equity: {initial_euqity}, Spread+Slippage: {spread}")


# **DATA**

In [ ]:
#add data stock
portfolio = {"ticker":[],"data":[]}
while len(portfolio["ticker"]) < 1:
  ticker_list = list(map(str, input("Enter a multiple ticker, space separated: ").split())) 
  for x in range(len(ticker_list)):
    if ticker_list[x] not in portfolio["ticker"]:
      # ticker_data = yf.Ticker(ticker_list[x]).history(period="max")
      ticker_data = pdr.get_data_yahoo(ticker_list[x], start=datetime.datetime(1900, 1, 1), end=datetime.datetime.now())
      if len(ticker_data) > 2:
        portfolio["ticker"].append(ticker_list[x])
        portfolio["data"].append(ticker_data)
      else:
        print("maaf data ticker "+ticker_list[x]+" tidak cukup data, minimal data 2000 baris")

In [ ]:
#cleaning and rekonstruk test data
start_date = datetime.datetime(1900, 1, 1)
end_date = datetime.datetime.now()
for i in range(len(portfolio["data"])):  
  if portfolio["data"][i].index.max() < end_date:
    end_date = portfolio["data"][i].index.max()
  if portfolio["data"][i].index.min() > start_date:
    start_date = portfolio["data"][i].index.min()
run_date = start_date
idx = []
data = []

bar = progressbar.ProgressBar(max_value=progressbar.UnknownLength)
i = 0
while run_date < end_date:  
  data_array = {} 
  for i in range(len(portfolio["data"])):   
    if run_date in portfolio["data"][i].index:
      data_array[portfolio["ticker"][i]] = portfolio["data"][i]["Adj Close"].loc[run_date]
    else:
      data_array[portfolio["ticker"][i]] = data[len(data)-1][portfolio["ticker"][i]]  
  idx.append(run_date)
  data.append(data_array)
  
  if run_date.weekday() == 4:
    run_date = run_date + datetime.timedelta(days=3)
  else:
    run_date = run_date + datetime.timedelta(days=1)

  #progress bar
  bar.update(i)
  i += 1
  if time.localtime().tm_sec == 10:
    break

test_data = pd.DataFrame(data,index=idx)
test_data.index.name = "date"
print('\n')  
print(test_data)

In [ ]:
from sys import stdout
from time import sleep
i =0
while i < 7500:
    stdout.write("\r%d" % i)
    stdout.flush()
    # sleep(0.01)
    i+=1
stdout.write("\n") # move the cursor to the next line

In [ ]:
from time import sleep
from tqdm import tqdm
import time
import sys

toolbar_width = 50

# setup toolbar
sys.stdout.write("[%s]" % (" " * toolbar_width))
sys.stdout.flush()
sys.stdout.write("\b" * (toolbar_width+1)) # return to start of line, after '['

# for i in tqdm(range(5)):
#     sleep(1)

time.sleep(0.1) # do real work here
sys.stdout.write("***")
sys.stdout.flush()
sys.stdout.write("]\n") # this ends the progress bar



In [ ]:
test_data = pd.DataFrame()
dff = []
data2 = { 
      portfolio["ticker"][0]:portfolio["data"][0]["Adj Close"].loc["2021-07-29"],
      portfolio["ticker"][1]:portfolio["data"][1]["Adj Close"].loc["2021-07-29"],
      portfolio["ticker"][2]:portfolio["data"][2]["Adj Close"].loc["2021-07-29"],
      portfolio["ticker"][3]:portfolio["data"][3]["Adj Close"].loc["2021-07-29"],
      portfolio["ticker"][4]:portfolio["data"][4]["Adj Close"].loc["2021-07-29"]
      }
dff.append(data2)
d = pd.DataFrame([data2],index=["2021-07-29"])
print(d)

In [ ]:
df = pd.DataFrame([data2],
                  index=["circle"])
df

In [ ]:
yf.pdr_override() # <== that's all it takes :-)

# download dataframe
data = pdr.get_data_yahoo("SPY", start="2017-01-01", end="2017-04-30")['Adj Close']
print(data)

In [ ]:
# creating an empty list
lst = []
# iterating till the range
for i in range(0, 12):
    ele = input(prompt=f'Ticker{i+1} ') 
    lst.append(ele) # adding the element     
print(f"portfolio {lst}")

In [ ]:
msft = yf.Ticker("MSFT")
hist = msft.history(period="max")
print(hist)

if len(hist) == 0:
  print("no data")

In [ ]:
pn.extension()
exchange = pn.widgets.Select(name='Select Exchange', options=['Exchange','NASDAQ', 'NYSE', 'LSE', 'EURONEXT'])
exchange

In [ ]:
# pn.extension()
# cross_selector = pn.widgets.CrossSelector(name='Fruits', value=['Apple', 'Pear'], options=['Apple', 'Banana', 'Pear', 'Strawberry'])
# cross_selector
# ticker1 = pn.widgets.MultiSelect(name='Select Tickers', value=[], options=['AAPL', 'AMD', 'AMZN', 'ATN'], size=8)

pn.extension()
ticker1 = pn.widgets.CrossSelector(name='Select Tickers', value=[], options=['AAPL', 'AMD', 'AMZN', 'ATN','AAPL', 'AMD', 'AMZN', 'ATN','AAPL', 'AMD', 'AMZN', 'ATN','AAPL', 'AMD', 'AMZN', 'ATN','AAPL', 'AMD', 'AMZN', 'ATN','AAPL', 'AMD', 'AMZN', 'ATN'], size=8)
ticker2 = pn.widgets.CrossSelector(name='Select Tickers', value=[], options=['AAPL2', 'AMD2', 'AMZN2', 'ATN2'], size=8)
ticker3 = pn.widgets.CrossSelector(name='Select Tickers', value=[], options=['AAPL3', 'AMD3', 'AMZN3', 'ATN3'], size=8)
ticker4 = pn.widgets.CrossSelector(name='Select Tickers', value=[], options=['AAPL4', 'AMD4', 'AMZN4', 'ATN4'], size=8)
if exchange.value == 'NASDAQ':
    display(ticker1)
elif exchange.value == 'NYSE':
    display(ticker2)
elif exchange.value == 'LSE':
    display(ticker3)
elif exchange.value == 'EURONEXT':
    display(ticker4)
else:
  print("no exchange select")
 

In [ ]:
print(ticker1.value[0])

In [ ]:
if len(ticker1.value) < 4:
  raise Exception("You dont have enough tickers")
else:
  df01 = pd.read_csv(f"https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/{ticker1.value[0]}.csv")
  df02 = pd.read_csv(f"https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/{ticker1.value[0]}.csv")
  df03 = pd.read_csv(f"https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/{ticker1.value[0]}.csv")
  df04 = pd.read_csv(f"https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/{ticker1.value[0]}.csv")
  df05 = pd.read_csv(f"https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/{ticker1.value[0]}.csv")
  df06 = pd.read_csv(f"https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/{ticker1.value[0]}.csv")
  df07 = pd.read_csv(f"https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/{ticker1.value[0]}.csv")
  df08 = pd.read_csv(f"https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/{ticker1.value[0]}.csv")
  df09 = pd.read_csv(f"https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/{ticker1.value[0]}.csv")
  df10 = pd.read_csv(f"https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/{ticker1.value[0]}.csv")
  df11 = pd.read_csv(f"https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/{ticker1.value[0]}.csv")
  df12 = pd.read_csv(f"https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/{ticker1.value[0]}.csv")
  df13 = pd.read_csv(f"https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/{ticker1.value[0]}.csv")
  df14 = pd.read_csv(f"https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/{ticker1.value[0]}.csv")
  df15 = pd.read_csv(f"https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/{ticker1.value[0]}.csv")
  df16 = pd.read_csv(f"https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/{ticker1.value[0]}.csv")
  df17 = pd.read_csv(f"https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/{ticker1.value[0]}.csv")
  df18 = pd.read_csv(f"https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/{ticker1.value[0]}.csv")
  df19 = pd.read_csv(f"https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/{ticker1.value[0]}.csv")
  df20 = pd.read_csv(f"https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/{ticker1.value[0]}.csv")
  df21 = pd.read_csv(f"https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/{ticker1.value[0]}.csv")
  df22 = pd.read_csv(f"https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/{ticker1.value[0]}.csv")
  df23 = pd.read_csv(f"https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/{ticker1.value[0]}.csv")
  df24 = pd.read_csv(f"https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/{ticker1.value[0]}.csv")
  df25 = pd.read_csv(f"https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/{ticker1.value[0]}.csv")
  df26 = pd.read_csv(f"https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/{ticker1.value[0]}.csv")
  df27 = pd.read_csv(f"https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/{ticker1.value[0]}.csv")
  df28 = pd.read_csv(f"https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/{ticker1.value[0]}.csv")
  df29 = pd.read_csv(f"https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/{ticker1.value[0]}.csv")
  df30 = pd.read_csv(f"https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/{ticker1.value[0]}.csv")

  print(df01.to_string()) 

In [ ]:
widgets.Combobox(
    # value='John',
    placeholder='Choose Someone',
    options=['Paul', 'John', 'George', 'Ringo'],
    description='Combobox:',
    ensure_option=True,
    disabled=False
)

In [ ]:
pn.extension()
progress = pn.indicators.Progress(name='Progress', value=20, width=200)
progress

In [ ]:
pn.extension()

terminal = pn.widgets.Terminal(
    "Welcome to the Panel Terminal!\nI'm based on xterm.js\n\n",
    options={"cursorBlink": True},
    height=300, sizing_mode='stretch_width'
)

terminal

In [ ]:
pn.extension()
large_df = pd._testing.makeCustomDataframe(100, 5) 

paginated_table = pn.widgets.Tabulator(large_df, disabled=True, pagination='remote', page_size=10)
paginated_table

In [ ]:
pn.extension()
df = pd.DataFrame({
    'int': [1, 2, 3],
    'float': [3.14, 6.28, 9.42],
    'str': ['A', 'B', 'C'],
    'bool': [True, False, True],
    'date': [dt.date(2019, 1, 1), dt.date(2020, 1, 1), dt.date(2020, 1, 10)]
}, index=[1, 2, 3])

df_widget = pn.widgets.Tabulator(df, disabled =True, configuration={"headerSort": False, "resizableColumns": False})
df_widget.servable()

In [ ]:
pn.extension()
json_obj = {   
    'data_id': "00000",
    'timestamp': "mm/dd/yyy 00:00:00",
    'buying_power': "No Data",
    'stock01_price': "No Data",
    'stock02_price': "No Data",
    'stock03_price': "No Data",
    'stock04_price': "No Data",
    'stock05_price': "No Data",
    'stock06_price': "No Data",
    'stock07_price': "No Data",
    'stock08_price': "No Data",
    'stock09_price': "No Data",
    'stock10_price': "No Data",
    'stock11_price': "No Data",
    'stock12_price': "No Data",
    'stock13_price': "No Data",
    'stock14_price': "No Data",
    'stock15_price': "No Data",
    'stock16_price': "No Data",
    'stock17_price': "No Data",
    'stock18_price': "No Data",
    'stock19_price': "No Data",
    'stock20_price': "No Data",
    'stock21_price': "No Data",
    'stock22_price': "No Data",
    'stock23_price': "No Data",
    'stock24_price': "No Data",
    'stock25_price': "No Data",
    'stock26_price': "No Data",
    'stock27_price': "No Data",
    'stock28_price': "No Data",
    'stock29_price': "No Data",
    'stock30_price': "No Data"
}

json = pn.pane.JSON(json_obj, name='JSON', height=300, width=500)

json

In [ ]:
pn.extension()
p1 = figure(width=300, height=300, name='Scatter', margin=5)
p1.scatter([0, 1, 2, 3, 4, 5, 6], [0, 1, 2, 3, 2, 1, 0])

p2 = figure(width=300, height=300, name='Line', margin=5)
p2.line([0, 1, 2, 3, 4, 5, 6], [0, 1, 2, 3, 2, 1, 0])

p3 = figure(width=300, height=300, name='Square', margin=5)
p3.square([0, 1, 2, 3, 4, 5, 6], [0, 1, 2, 3, 2, 1, 0], size=10)

accordion.append(p3)
accordion

In [ ]:
pn.extension()
number = pn.indicators.Number(
    name='Failure Rate', value=72, format='{value}%',
    colors=[(33, 'green'), (66, 'gold'), (100, 'red')]
)

pn.Row(number.clone(value=10), number.clone(value=42), number.clone(value=93))

In [ ]:
pn.extension()
data = {'x': np.arange(50), 'y': np.random.randn(50).cumsum()}

trend = pn.indicators.Trend(
    title='Price', data=data, width=200, height=200
)
trend
def stream_data():
    trend.stream({'x': [trend.data['x'][-1]+1], 'y': [trend.data['y'][-1]+np.random.randn()]}, rollover=50)

pn.state.add_periodic_callback(stream_data, period=250, count=100);
pn.Row(*(trend.clone(plot_type=pt) for pt in trend.param.plot_type.objects))

In [ ]:
pn.extension()
date_range_slider = pn.widgets.DateRangeSlider(
    name='Date Range Slider',
    start=dt.datetime(2017, 1, 1), end=dt.datetime(2019, 1, 1),
    value=(dt.datetime(2017, 1, 1), dt.datetime(2018, 1, 10))
)

date_range_slider

In [ ]:
from ipywidgets import Button
import asyncio

def wait_for_change(widget):
    future = asyncio.Future()
    def getvalue(change):
        future.set_result(change.description)
        widget.on_click(getvalue, remove=True) 
        # we need to free up the binding to getvalue to avoid an InvalidState error
        # buttons don't support unobserve
        # so use `remove=True` 
    widget.on_click(getvalue)
    return future

button = Button(description="wow")

list_to_tag = ["one", "two", "three", "four"]

async def f():
    for i in list_to_tag:
        print("going to tag {}".format(i))
        x = await wait_for_change(button)
        print("tagged {} with {}".format(i, x))
        print()

asyncio.create_task(f())
button

In [ ]:
pn.extension()

df = pd.DataFrame({'int': [1, 2, 3], 'float': [3.14, 6.28, 9.42], 'str': ['A', 'B', 'C'], 'bool': [True, False, True]})

df_widget = pn.widgets.DataFrame(df, name='DataFrame')

df_widget

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Table(
    header=dict(values=['A Scores', 'B Scores'],
                # line_color='darkslategray',
                # fill_color='lightskyblue',
                align='left'),
    cells=dict(values=[[100, 90, 80, 90,100, 90, 80, 90,100, 90, 80, 90,100, 90, 80, 90,100, 90, 80, 90], # 1st column
                       [95, 85, 75, 95,95, 85, 75, 95,95, 85, 75, 95,95, 85, 75, 95,95, 85, 75, 95]], # 2nd column
              #  line_color='darkslategray',
              #  fill_color='lightcyan',
               align='left'))
])

fig.update_layout(width=400, height=350)
fig.show()